In [2]:
import pandas as pd
#!conda install lxml --yes

Loading data from Wikipedia and getting appropriate table.

In [9]:
wiki_tables = pd.read_html(io="http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
nh_df_orig = wiki_tables[0]
nh_df = nh_df_orig

Removing rows with `Borough` `Not assigned` value.

In [16]:
nh_df = nh_df[nh_df["Borough"] != "Not assigned"]

Boroughs with multiple neighborhoods have them already listed in `Neighborhood` column, separated with slash.

No cells with `Neighborhood` empty or null.

In [30]:
nh_df[nh_df["Neighborhood"].isnull()]

,Postal code,Borough,Neighborhood


In [31]:
nh_df.shape

(103, 3)